<a href="https://colab.research.google.com/github/madegde/madegde.github.io/blob/main/big_data_lecture_practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Big Data

This notebook demonstrates use a large dataset of British price micro data.

The data possesses the characteristics of Big Data:

- Volume. The scale of data generated. Millions of rows (from a dataset of tens of millions) are presented.
- Velocity. The speed at which data is generated and processed in real time. Data is generated each day in real time.
- Variety. The diversity of data formats, from structured to unstructured, and dimensionality. Both numeric and text data is provided.


</br> </br>


In [1]:
import pandas as pd
import altair as alt

## Loading the data

Typically, you would load Big Data from a database or alternate source. Today, we will be reading (large) CSVs instead.

</br>
</br>

Two sources are provided:

- **Prices** Daily price observations for British supermarket products. The prices are identified according to via an ID for the store and product.
- **Items** Descriptive classification information for the products. The products are identified by their equivalent item in the CPI bsaket but their exact product name and store identity are anonymised.

In [2]:
prices_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/redacted_prices_df.csv')
items_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/redacted_items_df.csv')

In [3]:
prices_df.sample(5)

,date,price,unit_price,loyalty_price,original_price,store_id,product_id
2846989,2024-02-22,3.25,14.44 per kg,NaN,NaN,3,145299.0
2929597,2024-02-29,11.50,3.66 per 100g,NaN,NaN,3,135305.0
2389158,2023-12-19,3.10,£1.48/100g,NaN,4.1,2,85292.0
4008017,2024-06-21,2.75,4.17 per litre,NaN,NaN,3,240270.0
2640293,2024-02-07,2.49,6.23 per 100g,NaN,NaN,3,137254.0


In [4]:
items_df.sample(5)

,store_id,product_id,cpi_id,cpi_name
2433,4,23681,212720.0,strawberries per kg or punnet
9071,3,96105,211901.0,tea bags pkt of 80 (230g-250g)
9268,6,98878,212905.0,vegetable pickle 280-520g
7770,5,78465,213006.0,coffee pods pack 8-16
9283,7,99134,212008.0,lemonade-2 litre bottle


# Bread Prices

What are we making?
- A line chart
- X: date (month? week?)
- Y: Price
- Line: One line for each type of product

Data:
Aggregated to date, product_type, price
- Median price for each product type
- Group by date, product_type

In [5]:
targets = ['chilled garlic bread', 'six bread rolls-white/brown', 'white sliced loaf branded 750g', 'wholemeal sliced loaf branded']

df = pd.merge(prices_df, items_df, on=['store_id', 'product_id'], how='inner')
df = df[df.cpi_name.isin(targets)]

df = df.sort_values("date")
df= df.drop_duplicates(keep="last")

df = df.groupby(by=["cpi_id", "cpi_name"]).agg({"price": "median"})
df


,,price
cpi_id,cpi_name,
210106.0,six bread rolls-white/brown,1.20
210111.0,white sliced loaf branded 750g,1.39
210113.0,wholemeal sliced loaf branded,1.40
210114.0,chilled garlic bread,1.65


In [6]:
targets = ['chilled garlic bread', 'six bread rolls-white/brown', 'white sliced loaf branded 750g', 'wholemeal sliced loaf branded']

In [7]:
df = pd.merge(prices_df, items_df, on=['store_id', 'product_id'], how='inner')
df = df[df.cpi_name.isin(targets)]

df['date'] = pd.to_datetime(df.date)

df['year'] = df.date.dt.year
df['month']=df.date.dt.month

df['year_month'] = df.year.astype(str)+"-"+df.month.astype(str)

df = df.groupby(by=["year_month", "cpi_id", "cpi_name"]).agg({"price": "median"})
df = df.reset_index()

df = df.sort_values(by='year_month')

first_vals = df.drop_duplicates(subset=["cpi_id"])
first_vals = first_vals[["cpi_id", "price"]]
first_vals.columns = ["cpi_id", "start_price"]
first_vals

df = df.merge(first_vals, on="cpi_id", how="left")
df['index'] = (df['price']/df['start_price'])*100


chart = alt.Chart(df).mark_line(interpolate="basis").encode(
    x=alt.X("year_month:T"),
    y=alt.Y("index:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("cpi_name:N")
)

chart.save("bread_chart.json", indent=4)

<ipython-input-7-724a3c9f3995>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df.date)
<ipython-input-7-724a3c9f3995>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df.date.dt.year
<ipython-input-7-724a3c9f3995>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [8]:
df

,year_month,cpi_id,cpi_name,price,start_price,index
0,2023-10,210106.0,six bread rolls-white/brown,1.19,1.19,100.000000
1,2023-10,210111.0,white sliced loaf branded 750g,1.40,1.40,100.000000
2,2023-10,210113.0,wholemeal sliced loaf branded,1.39,1.39,100.000000
3,2023-10,210114.0,chilled garlic bread,1.70,1.70,100.000000
4,2023-11,210106.0,six bread rolls-white/brown,1.20,1.19,100.840336
...,...,...,...,...,...,...
67,2024-8,210114.0,chilled garlic bread,1.60,1.70,94.117647
68,2024-9,210111.0,white sliced loaf branded 750g,0.75,1.40,53.571429
69,2024-9,210113.0,wholemeal sliced loaf branded,1.50,1.39,107.913669
70,2024-9,210106.0,six bread rolls-white/brown,1.19,1.19,100.000000


In [9]:
items_df.query("cpi_id == 210111.0")

,store_id,product_id,cpi_id,cpi_name
1961,4,22263,210111.0,white sliced loaf branded 750g
2272,4,23296,210111.0,white sliced loaf branded 750g
2525,4,23844,210111.0,white sliced loaf branded 750g
3222,4,26255,210111.0,white sliced loaf branded 750g
4324,7,33813,210111.0,white sliced loaf branded 750g
6053,7,56224,210111.0,white sliced loaf branded 750g
8522,6,88550,210111.0,white sliced loaf branded 750g
9973,7,33533,210111.0,white sliced loaf branded 750g
15004,7,174486,210111.0,white sliced loaf branded 750g
16889,6,200224,210111.0,white sliced loaf branded 750g


In [10]:
items_df[items_df.cpi_name.str.contains("bag", case=False)].cpi_name.unique()

array(['popcorn, bag, 10-30g', 'tea bags pkt of 80 (230g-250g)',
       'flavrd tea 15-20 bags, 30-40g', 'fresh veg-cabbage-whole-per kg',
       'brand choc sweets 90-185g bag', 'sweets, not choc, 150-250g bag',
       'tea bags-2-packet of 240'], dtype=object)

In [11]:
targets = ['chilled garlic bread', 'six bread rolls-white/brown', 'white sliced loaf branded 750g', 'wholemeal sliced loaf branded']

</br>
</br>
</br>


# Associating the dataframes

Our `prices_df` contains prices and ids for the store (`store_id`) and product (`product_id`) but it would be easier to work from a dataframe that includes product informaton as well, which is contained in `items_df`.

</br></br>

Let's associate the data with a merge.

In [12]:
df = pd.merge(prices_df, items_df, on=['store_id', 'product_id'], how='inner')

</br></br>

# Investigating the data

Let's take a look at the data we have.

</br></br>
</br></br>


## Stores

How do prices vary across store? Let's find out.

In [13]:
df

,date,price,unit_price,loyalty_price,original_price,store_id,product_id,cpi_id,cpi_name
0,2023-10-06,12.95,0.16 per 100ml,NaN,NaN,5,209870.0,212023.0,cola/fizzy drink 330ml pk 4-8
1,2023-10-06,12.95,0.16 per 100ml,NaN,NaN,5,209870.0,212025.0,"cola drink, reg,bottle,1.25-2l"
2,2023-10-06,9.00,9 per 75cl,NaN,NaN,5,265800.0,310426.0,sparkling wine 75cl min 11%abv
3,2023-10-06,4.00,1.29 per 100g,NaN,NaN,5,181052.0,212228.0,malted chocolate sweets
4,2023-10-06,4.00,1.29 per 100g,NaN,NaN,5,181052.0,212218.0,carton/box of chocs 150-400gm
...,...,...,...,...,...,...,...,...,...
6443978,2024-05-06,2.75,£4.44 per 1 litre,NaN,NaN,7,163496.0,310220.0,spec'y beer bott 500ml 4-5.5
6443979,2024-05-06,2.75,£4.44 per 1 litre,NaN,NaN,7,163496.0,310111.0,bottled premium lager 4.3-7.5%
6443980,2024-05-06,2.75,£4.44 per 1 litre,NaN,NaN,7,163496.0,310217.0,lager 10-24 bottles 250-330ml
6443981,2024-04-08,10.00,£2.53/lt,NaN,NaN,1,62053.0,310215.0,lager 4 bottles- premium


In [15]:
store_prices = df.copy()

median_prices = store_prices.groupby(['store_id']).agg({"price": ["median", "mean"]})

median_prices = median_prices.reset_index()

median_prices.columns = ['store_id', 'median', 'mean']

median_prices = median_prices.melt(id_vars='store_id', value_vars=['median', 'mean'], var_name='price_type', value_name='price') # Going from wide to long format

median_prices

,store_id,price_type,price
0,1,median,2.350000
1,2,median,3.000000
2,3,median,2.490000
3,4,median,1.490000
4,5,median,2.500000
5,6,median,2.650000
6,7,median,2.500000
7,1,mean,4.357184
8,2,mean,7.716530
9,3,mean,5.225812


In [16]:
store_prices = df.copy()

median_prices = store_prices.groupby(['store_id']).agg({'price': ['median', 'mean']})

median_prices = median_prices.reset_index()

median_prices.columns = ['store_id', 'median', 'mean']

median_prices

,store_id,median,mean
0,1,2.35,4.357184
1,2,3.00,7.716530
2,3,2.49,5.225812
3,4,1.49,2.280244
4,5,2.50,4.833674
5,6,2.65,5.032822
6,7,2.50,3.677910


In [17]:
store_prices = df.copy()

median_prices = store_prices.groupby(['store_id']).agg({'price': ['median', 'mean']})
median_prices = median_prices.reset_index()
median_prices.columns = ['store_id', 'median_price', 'mean_price']

median_prices = median_prices.melt(id_vars='store_id', value_vars=['median_price', 'mean_price'], var_name='price_type', value_name='price') # Going from wide to long format


alt.Chart(median_prices).mark_bar().encode(
    column=alt.Column('store_id', title=''),
    x=alt.X('price_type', title='', axis=alt.Axis(labels=False)),
    y=alt.Y('price', title='', axis={"labelExpr": "'£' + datum.label", "labelOverlap": False}),
    color='price_type'
).properties(
    title = {
        'text': "Prices by store",
        'subtitle': ["Mean and median prices", ""]
    },
    width=100)


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Let's make a grouped bar chart of this

In [19]:
median_prices = median_prices.melt(id_vars='store_id', value_vars=['median', 'mean'], var_name='price_type', value_name='price') # Going from wide to long format

median_prices['store_id'] = "Store " + median_prices['store_id'].astype(str) # Adding 'Store' to store_id for nicer labels

alt.Chart(median_prices).mark_bar().encode(
    column=alt.Column('store_id', title=''),
    x=alt.X('price_type', title='', axis=alt.Axis(labels=False)),
    y=alt.Y('price', title='', axis={"labelExpr": "'£' + datum.label", "labelOverlap": False}),
    color='price_type'
).properties(
    title = {
        'text': "Prices by store",
        'subtitle': ["Mean and median prices", ""]
    },
    width=100)



ValueError: value_name (price) cannot match an element in the DataFrame columns.

### <b> Items </b>

What about items? Can we tell which are the most expensive types of products sold in supermarkets?

In [20]:
df

,date,price,unit_price,loyalty_price,original_price,store_id,product_id,cpi_id,cpi_name
0,2023-10-06,12.95,0.16 per 100ml,NaN,NaN,5,209870.0,212023.0,cola/fizzy drink 330ml pk 4-8
1,2023-10-06,12.95,0.16 per 100ml,NaN,NaN,5,209870.0,212025.0,"cola drink, reg,bottle,1.25-2l"
2,2023-10-06,9.00,9 per 75cl,NaN,NaN,5,265800.0,310426.0,sparkling wine 75cl min 11%abv
3,2023-10-06,4.00,1.29 per 100g,NaN,NaN,5,181052.0,212228.0,malted chocolate sweets
4,2023-10-06,4.00,1.29 per 100g,NaN,NaN,5,181052.0,212218.0,carton/box of chocs 150-400gm
...,...,...,...,...,...,...,...,...,...
6443978,2024-05-06,2.75,£4.44 per 1 litre,NaN,NaN,7,163496.0,310220.0,spec'y beer bott 500ml 4-5.5
6443979,2024-05-06,2.75,£4.44 per 1 litre,NaN,NaN,7,163496.0,310111.0,bottled premium lager 4.3-7.5%
6443980,2024-05-06,2.75,£4.44 per 1 litre,NaN,NaN,7,163496.0,310217.0,lager 10-24 bottles 250-330ml
6443981,2024-04-08,10.00,£2.53/lt,NaN,NaN,1,62053.0,310215.0,lager 4 bottles- premium


In [21]:
item_prices = df.groupby(by=['cpi_id', 'cpi_name']).agg({'price': 'median'})

item_prices.sort_values(by=["price"], ascending=False)

,,price
cpi_id,cpi_name,
310423.0,bottle of champagne 75 cl,39.99
310401.0,whisky-70 cl bottle,28.00
320206.0,hand rolling tobacco pack 30gm,20.05
310429.0,gin bottle 70cl,20.00
310403.0,vodka-70 cl bottle,19.00
...,...,...
210221.0,couscous plain/flavoured,1.15
212613.0,pulses can 390 - 420g,1.10
210324.0,crumpets pack 6-9 spec number,1.10


In [22]:
item_prices = df.groupby(by=['cpi_id', 'cpi_name']).agg({'price': ['median', 'var']})
item_prices = item_prices.reset_index()
item_prices.columns = ['cpi_id', 'cpi_name', 'price_median', 'price_var']
item_prices.sort_values(by='price_var', ascending=False)

,cpi_id,cpi_name,price_median,price_var
190,310423.0,bottle of champagne 75 cl,39.99,1301.450305
183,310315.0,bottle of wine 70-75cl,7.00,937.354068
197,320108.0,cigarettes 8,14.15,618.753449
25,210416.0,beef roasting joint per kg,11.00,560.348414
198,320115.0,cigarettes 15,14.20,515.736353
...,...,...,...,...
34,211007.0,canned meat-stewed steak,2.45,0.272655
1,210111.0,white sliced loaf branded 750g,1.39,0.167294
33,210914.0,fresh turkey diced/min per kg,3.09,0.113608
27,210703.0,home killed pork-loin chops kg,4.25,0.106801


In [ ]:
# EX1: Try to calculate the average price of items

# HINT: try grouping by cpi_id instead of store_id


# EX2: Which products have the highest/least variance? (hint: agg with 'var')


In [24]:
avg_prices = df.groupby(by=['cpi_id', 'cpi_name']).agg({'price': ['mean', 'var']})
avg_prices = avg_prices.reset_index()
avg_prices.columns = ['cpi_id', 'cpi_name', 'price_mean', 'price_var']
avg_prices.sort_values(by='price_var', ascending=False)

,cpi_id,cpi_name,price_mean,price_var
190,310423.0,bottle of champagne 75 cl,44.629263,1301.450305
183,310315.0,bottle of wine 70-75cl,13.654412,937.354068
197,320108.0,cigarettes 8,25.657560,618.753449
25,210416.0,beef roasting joint per kg,16.253124,560.348414
198,320115.0,cigarettes 15,24.527751,515.736353
...,...,...,...,...
34,211007.0,canned meat-stewed steak,2.568075,0.272655
1,210111.0,white sliced loaf branded 750g,1.077767,0.167294
33,210914.0,fresh turkey diced/min per kg,3.287075,0.113608
27,210703.0,home killed pork-loin chops kg,4.271497,0.106801


</br></br>
</br></br>

### <b>Price distributions</b>

What does the price distribution of our dataset look like?

In [25]:
df.price.describe()

,price
count,6.443983e+06
mean,4.934667e+00
std,9.388128e+00
min,1.000000e-02
25%,1.500000e+00
50%,2.500000e+00
75%,4.150000e+00
max,3.000000e+02


Can we display this more intuitively? Let's make a histogram.

Let's show prices in 10p bins from £0-10

In [26]:
hist_df = prices_df.copy()

hist_df['rounded_price'] = hist_df['price'].round(1)

hist_df = hist_df.groupby('rounded_price').agg({'price': 'count'}).reset_index()

hist_df = hist_df.query("rounded_price <= 10")

# Rename the columns for clarity: 'rounded_price' to 'price', and the count to 'density'
hist_df.columns = ['price', 'density']

hist_df

,price,density
0,0.0,18
1,0.1,281
2,0.2,2821
3,0.3,5702
4,0.4,28834
...,...,...
96,9.6,909
97,9.7,571
98,9.8,2926
99,9.9,274


In [27]:
# Create a copy of the original DataFrame
hist_df = prices_df.copy()

# Round the 'price' column to 1 decimal place to group prices into rounded intervals
hist_df['rounded_price'] = hist_df['price'].round(1)

# Group by the rounded prices and count the occurrences of each rounded price
hist_df = hist_df.groupby('rounded_price').agg({'price': 'count'}).reset_index()

# Filter out rows where the rounded price is greater than 10
hist_df = hist_df.query("rounded_price <= 10")

# Rename the columns for clarity: 'rounded_price' to 'price', and the count to 'density'
hist_df.columns = ['price', 'density']

# Normalize the density values to calculate the relative frequency (density)
hist_df['density'] = hist_df['density'] / hist_df['density'].sum()

# Create a histogram using Altair
histogram = alt.Chart(hist_df).mark_bar(
    width=5
).encode(
    x=alt.X('price:Q',  title='', axis={"labelExpr": "'£'+datum.value"}),  # Bin the 'price' values into 20 bins
    y=alt.Y('density:Q', title='Density'),  # Plot the normalized density on the y-axis,
    tooltip=['price', 'density']  # Show the 'price' and 'density' values on hover
)

histogram


alt.Chart(...)

In [28]:
# Create a copy of the original DataFrame
hist_df = prices_df.copy()

# Round the 'price' column to 1 decimal place to group prices into rounded intervals
hist_df['rounded_price'] = hist_df['price'].round(1)

# Group by the rounded prices and count the occurrences of each rounded price
hist_df = hist_df.groupby('rounded_price').agg({'price': 'count'}).reset_index()

# Filter out rows where the rounded price is greater than 10
hist_df = hist_df.query("rounded_price <= 10")

# Rename the columns for clarity: 'rounded_price' to 'price', and the count to 'density'
hist_df.columns = ['price', 'density']

# Normalize the density values to calculate the relative frequency (density)
hist_df['density'] = hist_df['density'] / hist_df['density'].sum()

# Create a histogram using Altair
histogram = alt.Chart(hist_df).mark_bar(
    width=5
).encode(
    x=alt.X('price:Q',  title='', axis={"labelExpr": "'£'+datum.value"}),  # Bin the 'price' values into 20 bins
    y=alt.Y('density:Q', title='Density'),  # Plot the normalized density on the y-axis,
    tooltip=['price', 'density']  # Show the 'price' and 'density' values on hover
)

# Display the histogram
histogram

alt.Chart(...)

</br> </br>

This is interesting. Can we Copy-Paste this code to loop over all our stores?

In [29]:
for store_id in prices_df.store_id.unique():
    temp_df = prices_df.query(f"store_id == {store_id}")
    # TODO: repeat the histogram code to create your own
    hist_df = temp_df
    hist_df['rounded_price'] = hist_df['price'].round(1)

    # Group by the rounded prices and count the occurrences of each rounded price
    hist_df = hist_df.groupby('rounded_price').agg({'price': 'count'}).reset_index()

    # Filter out rows where the rounded price is greater than 10
    hist_df = hist_df.query("rounded_price <= 10")

    # Rename the columns for clarity: 'rounded_price' to 'price', and the count to 'density'
    hist_df.columns = ['price', 'density']

    # Normalize the density values to calculate the relative frequency (density)
    hist_df['density'] = hist_df['density'] / hist_df['density'].sum()

    # Create a histogram using Altair
    histogram = alt.Chart(hist_df).mark_bar(
        width=5
    ).encode(
        x=alt.X('price:Q',  title='', axis={"labelExpr": "'£'+datum.value"}),  # Bin the 'price' values into 20 bins
        y=alt.Y('density:Q', title='Density'),  # Plot the normalized density on the y-axis,
        tooltip=['price', 'density']  # Show the 'price' and 'density' values on hover
    ).properties(
        title={
            "text": f"Store {store_id} price distribution"
        }
    )
    histogram.display()



<ipython-input-29-43013ed8f2ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_df['rounded_price'] = hist_df['price'].round(1)


alt.Chart(...)

<ipython-input-29-43013ed8f2ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_df['rounded_price'] = hist_df['price'].round(1)


alt.Chart(...)

<ipython-input-29-43013ed8f2ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_df['rounded_price'] = hist_df['price'].round(1)


...

alt.Chart(...)

<ipython-input-29-43013ed8f2ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_df['rounded_price'] = hist_df['price'].round(1)


alt.Chart(...)

<ipython-input-29-43013ed8f2ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_df['rounded_price'] = hist_df['price'].round(1)


alt.Chart(...)

</br> </br>

### <b> A specific example: Olive Oil </b>

Olive oil h

In [30]:
items_df[items_df.cpi_name.str.contains("olive", case=False)]

,store_id,product_id,cpi_id,cpi_name
169,1,1528,211408.0,olive oil - 500ml - 1 litre
202,1,1794,211408.0,olive oil - 500ml - 1 litre
325,1,2732,211408.0,olive oil - 500ml - 1 litre
328,1,2738,211408.0,olive oil - 500ml - 1 litre
338,1,2901,211408.0,olive oil - 500ml - 1 litre
...,...,...,...,...
22024,2,267919,211408.0,olive oil - 500ml - 1 litre
22059,6,268525,211408.0,olive oil - 500ml - 1 litre
22061,2,268579,211408.0,olive oil - 500ml - 1 litre
22543,5,275134,211408.0,olive oil - 500ml - 1 litre


In [31]:
olive_oil_df = df.query("cpi_id == 211408.0") # Filtering for just Olive Oil
olive_oil_df

,date,price,unit_price,loyalty_price,original_price,store_id,product_id,cpi_id,cpi_name
1360,2023-10-06,6.85,0.68 per 100ml,NaN,NaN,5,156473.0,211408.0,olive oil - 500ml - 1 litre
1433,2023-10-06,7.00,0.7 per 100ml,NaN,NaN,5,100834.0,211408.0,olive oil - 500ml - 1 litre
1455,2023-10-06,5.10,1.02 per 100ml,NaN,NaN,5,192970.0,211408.0,olive oil - 500ml - 1 litre
1456,2023-10-06,2.80,1.12 per 100ml,NaN,NaN,5,201425.0,211408.0,olive oil - 500ml - 1 litre
1497,2023-10-06,7.75,0.78 per 100ml,NaN,NaN,5,257913.0,211408.0,olive oil - 500ml - 1 litre
...,...,...,...,...,...,...,...,...,...
6442990,2024-04-10,7.00,£1.40/100ml,NaN,NaN,1,227285.0,211408.0,olive oil - 500ml - 1 litre
6443037,2024-05-15,6.50,£1.30 / 100ml,NaN,6.50,6,65445.0,211408.0,olive oil - 500ml - 1 litre
6443238,2024-08-30,5.75,£1.15 / 100ml,NaN,5.75,6,221602.0,211408.0,olive oil - 500ml - 1 litre
6443727,2024-05-02,10.00,£2.00 / 100ml,NaN,10.00,6,78258.0,211408.0,olive oil - 500ml - 1 litre


Does Olive Oil cost more at some places than others?
Let's check final prices and see

In [32]:
final_prices = olive_oil_df.drop_duplicates(subset=['store_id', 'product_id'], keep='last') # Keeping the last price for each store-product pair
final_prices['store_id'] = "Store " + final_prices['store_id'].astype(str) # Adding 'Store' to store_id for nicer labels

alt.Chart(final_prices).mark_circle(size=100).encode(
    y=alt.Y('store_id:N', title=''),
    x=alt.X('price:Q', title='Price (£)'),
    color=alt.Color('store_id:N', legend=None),
).properties(
    width=500,
    height=400,
    title={
        'text': "Olive Oil prices by store",
        'subtitle': ["Most recent price for each product", ""],
        'anchor': 'start',
    }
)


<ipython-input-32-ba437aed987d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_prices['store_id'] = "Store " + final_prices['store_id'].astype(str) # Adding 'Store' to store_id for nicer labels
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


alt.Chart(...)